Import libraries

In [ ]:
# import system libraries
import os
import time
import itertools
import shutil
import pathlib

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import math
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, LearningRateScheduler

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('required modules are loaded')

Load train and test dataset

In [ ]:
train_csv_file = pd.read_csv('./Training_set.csv')
test_csv_file = pd.read_csv('./Testing_set.csv')

Create fucntions for evaluate the dataset and call functions to get an idea about the dataset

In [ ]:
def shape_of_the_dataset(dataframe, dataset_name='dataframe'):
    print(f"{dataset_name} dataset has {dataframe.shape[0]} rows and {dataframe.shape[1]} columns")
    
def check_null_values(dataframe, dataset_name='dataframe'):
    print(f"Number of null values in the {dataset_name} dataset: ")
    print(dataframe.isnull().sum())

def check_unique_values(dataframe, dataset_name='dataframe'):
    print(f"Number of unique values in {dataset_name} dataset: ")
    print(dataframe.nunique())
    
def seperator(sep=50):
    print("-"*sep)

In [ ]:
train_csv_file.head()

In [ ]:
test_csv_file.head()

In [ ]:
shape_of_the_dataset(train_csv_file, "Train")
shape_of_the_dataset(test_csv_file, "Test")

In [ ]:
check_null_values(train_csv_file, 'Train')

In [ ]:
check_null_values(test_csv_file, 'Test')

In [ ]:
check_unique_values(train_csv_file, 'Train')

In [ ]:
check_unique_values(test_csv_file, 'Test')

In [ ]:
msno.matrix(train_csv_file)
plt.title('Missing Values Distribution in the Training dataset', fontsize=28, fontstyle='oblique');

In [ ]:
msno.matrix(est_csv_file)
plt.title('Missing Values Distribution in the Testing dataset', fontsize=28, fontstyle='oblique');

In [ ]:
train_csv_file['filename'] = './dataset/train/' + train_csv_file['filename']
test_csv_file['filename'] = './dataset/test/' + test_csv_file['filename']

Train and test data set split

In [ ]:
train_dataset_size = 0.80
train_dataframe, valid_dataframe = train_test_split(train_csv_file,  train_size= train_dataset_size, shuffle= True, random_state= 123)

In [ ]:
train_dataframe.head()

In [ ]:
valid_dataframe.head()

In [ ]:
shape_of_the_dataset(train_dataframe, "Train")
shape_of_the_dataset(valid_dataframe, "Valid")

In [ ]:
check_null_values(train_dataframe, "Train")

In [ ]:
check_null_values(valid_dataframe, "Valid")

In [ ]:
check_unique_values(train_dataframe, "Train")

In [ ]:
check_unique_values(valid_dataframe, "Valid")

In [ ]:
def class_count_plot(x, dataframe, title, xlabel, ylabel, width, height, order = None, rotation=False, palette='winter', hue=None):
    count = len(dataframe)
    plt.figure(figsize=(width,height))
    ax = sns.countplot(x = x, palette=palette, order = order, hue=hue)
    plt.title(title, fontsize=20)
    if rotation:
        plt.xticks(rotation = 'vertical')
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.show()

In [ ]:
train_ds_order = train_dataframe['label'].value_counts()
train__ds_order

In [ ]:
x = train_dataframe['label']
class_count_plot(x, train_dataframe, "Class Distrbution in the Training dataset", 'Class', 'Frequency', 15,10, order = train_order.index, rotation=True)

In [ ]:
valid_ds_order = valid_dataframe['label'].value_counts()
valid_ds_order

In [ ]:
x = valid_dataframe['label']
class_count_plot(x, valid_dataframe, "Class Distrbution in the Validation dataset", 'Class', 'Frequency', 15,10, order = valid_order.index, rotation=True, palette='autumn')

In [ ]:
batch_size = 16
image_size = (224, 224)
channels = 3
image_shape = (image_size[0], image_size[1], channels)

test_length = len(test_csv_file)
test_batch_size = max(sorted([test_length // n for n in range(1, test_length + 1) if test_length%n == 0 and test_length/n <= 80]))
total_test_steps = ts_length // test_batch_size

print(test_length)
print(test_batch_size)
print(total_test_steps)

In [ ]:
train_dataframe['filename'][0]

data augmentation

In [ ]:
def scalar(img):
    return img

t_gen = ImageDataGenerator(preprocessing_function= scalar,
                            rotation_range=40,
                            width_shift_range=0.3,
                            height_shift_range=0.2,
                            brightness_range=None,
                            shear_range=0.1,
                            zoom_range=0.3,
                            channel_shift_range=0.4)

tst_gen = ImageDataGenerator(preprocessing_function= scalar,
                            rotation_range=40,
                            width_shift_range=0.3,
                            height_shift_range=0.2,
                            brightness_range=None,
                            shear_range=0.1,
                            zoom_range=0.3,
                            channel_shift_range=0.4)

train_gen = t_gen.flow_from_dataframe( train_df, 
                                        x_col= 'filename', 
                                        y_col= 'label', 
                                        target_size= img_size, 
                                        class_mode= 'categorical',
                                        color_mode= 'rgb', 
                                        shuffle= False, 
                                        batch_size= batch_size)

valid_gen = tst_gen.flow_from_dataframe( valid_df, 
                                       x_col= 'filename', 
                                       y_col= 'label', 
                                       target_size= img_size, 
                                       class_mode= 'categorical',
                                       color_mode= 'rgb', 
                                       shuffle= False, 
                                       batch_size= batch_size)
test_gen = tst_gen.flow_from_dataframe( test_csv_file, 
                                       x_col= 'filename', 
                                       y_col= 'label', 
                                       target_size= img_size, 
                                       class_mode= 'categorical',
                                       color_mode= 'rgb', 
                                       shuffle= False, 
                                       batch_size= batch_size)


In [ ]:
get_dic = train_gen.class_indices      # defines dictionary {'class': index}
classes = list(get_dic.keys())       # defines list of dictionary's kays (classes), classes names : string
imgs, lbls = next(train_gen)      # get a batch size samples from the generator

plt.figure(figsize= (20, 20))

for i in range(24):
    plt.subplot(4, 4, i + 1)
    img = imgs[i] / 255       # scales data to range (0 - 255)
    plt.imshow(img)
    index = np.argmax(lbls[i])  # get image index
    class_name = classes[index]   # get class of image
    plt.title(class_name, color= 'blue', fontsize= 12)
    plt.axis('off')
plt.show()

Create model

In [ ]:
# Create Model Structure
#from keras.applications import EfficientNetB0
image_size = (224, 224)
channels = 3
image_shape = (image_size[0], image_size[1], channels)
class_count = len(list(train_gen.class_indices.keys())) # to define number of classes in dense layer

# create pre-trained model (you can built on pretrained model such as :  efficientnet, VGG , Resnet )
base_model = tf.keras.applications.efficientnet.EfficientNetB5(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')
#base_model = efc.EfficientNetB5(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')
base_model.trainable = False

model = Sequential([
    base_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(512, activation = 'relu'),
    Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])
model.build([None, 224, 224, 3])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

define early stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', 
                               patience=5, 
                               restore_best_weights=True,
                               mode='max',
                              )

def step_decay(epoch):
    
     initial_lrate = 0.1
     drop = 0.5
     epochs_drop = 10.0
     learning_rate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
     return learning_rate

learn__rate_scheduler = LearningRateScheduler(step_decay)

run the model

In [ ]:
batch_size = 16   # set batch size for training
epochs = 10   # number of all epochs in training

history = model.fit(x= train_gen, 
                    epochs= epochs, 
                    verbose= 1, 
                    validation_data= valid_gen, 
                    validation_steps= None, 
                    shuffle= False,
                    batch_size = batch_size,
                    callbacks=[early_stopping])

evaluate training of the model 

In [ ]:
# Define needed variables
train_accuracy = history.history['accuracy']
train_loss = history.history['loss']
validation_accuracy = history.history['val_accuracy']
validation_loss = history.history['val_loss']
index_loss = np.argmin(validation_loss)
validation_lowest = validation_loss[index_loss]
index_accuracy = np.argmax(validation_accuracy)
accuracy_highest = validation_accuracy[index_accuracy]
Epochs = [i+1 for i in range(len(train_accuracy))]
loss_label = f'best epoch= {str(index_loss + 1)}'
accuracy_label = f'best epoch= {str(index_accuracy + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, train_loss, 'r', label= 'Training loss')
plt.plot(Epochs, validation_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, validation_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, train_accuracy, 'r', label= 'Training Accuracy')
plt.plot(Epochs, validation_accuracy, 'g', label= 'Validation Accuracy')
plt.scatter(index_accuracy + 1 , accuracy_highest, s= 150, c= 'blue', label= accuracy_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

In [ ]:
training_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
validation_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", training_score[0])
print("Train Accuracy: ", training_score[1])
print('-' * 20)
print("Validation Loss: ", validation_score[0])
print("Validation Accuracy: ", validation_score[1])

save the model

In [ ]:
model.save("butterfly_efficientnet.h5")

evaluate the model on test dataset

In [ ]:
test_loss, test_accuracy = model.evaluate(test_gen)
print(f'test accuracy : {test_accuracy * 100:.2f}%')

In [ ]:
def prediction_on_test_data(test_gen):    
    y_prediction= []
    error_list=[]
    error_prediction_list = []
    y_true=test_gen.labels
    classes=list(test_gen.class_indices.keys())
    class_count=len(classes)
    errors=0
    predictions=model.predict(test_gen, verbose=1)
    tests=len(predictions)    
    for i, p in enumerate(predictions):        
        prediction_index=np.argmax(p)         
        true_index=test_gen.labels[i]  # labels are integer values        
        if prediction_index != true_index: # a misclassification has occurred                                           
            errors=errors + 1
            file=test_gen.filenames[i]
            error_list.append(file)
            error_class=classes[prediction_index]
            error_prediction_list.append(error_class)
        y_prediction.append(prediction_index)
            
    accuracy=( 1-errors/tests) * 100
    message=f'there were {errors} errors in {tests} tests for an accuracy of {acc:6.2f}'
    print(message) 
    yprediction=np.array(y_prediction)
    ytrue=np.array(y_true)
    f1score=f1_score(ytrue, yprediction, average='weighted')* 100
    if class_count <=75:
        cm = confusion_matrix(ytrue, yprediction )
        # plot the confusion matrix
        plt.figure(figsize=(500, 500))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
        plt.xticks(np.arange(class_count)+.5, classes, rotation=90)
        plt.yticks(np.arange(class_count)+.5, classes, rotation=0)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
    classification_rep = classification_report(y_true, y_pred, target_names=classes, digits= 4) # create classification report
    print("Classification Report:\n----------------------\n", classification_rep)
    return errors, tests, error_list, error_prediction_list, f1score

errors, tests, error_list, error_prediction_list, f1score =prediction_on_test_data(test_gen)